<a href="https://colab.research.google.com/github/Aluminiy/hw01/blob/main/Environment_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [3]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [4]:
%%writefile paper_agent.py
def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [5]:
%%writefile scissors_agent.py
def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [6]:
%%writefile random_agent.py
def random_agent(observation, configuration):
    return random.randint(0, 2)

Writing random_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [7]:
%%writefile copy_opponent.py

#Example 
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


In [8]:
%%writefile reactionary.py

def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Writing reactionary.py


In [9]:
%%writefile statistical.py

action_histogram = {}


def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]

In [ ]:
evaluate(
    "rps",
    ["rock_agent.py", "random_agent.py"],
    configuration={"episodeSteps": 100}
)

[[1, None]]

In [ ]:
evaluate(
    "rps",
    ["random_agent.py", "statistical.py"],
    configuration={"episodeSteps": 100}
)

[[None, 1]]

In [21]:
agents=['rock_agent.py', 'paper_agent.py', 'scissors_agent.py', 'random_agent.py', 'copy_opponent.py', 'reactionary.py', 'statistical.py']

scores = np.zeros((len(agents), len(agents), 2))

for i, agent1 in enumerate(agents):
    for j, agent2 in enumerate(agents):
        if i == j:
            continue
        res = evaluate("rps", [agent1, agent2], configuration={"episodeSteps": 100})
        if (res[0][0] != None):
            scores[i][j][0] = res[0][0]
        if (res[0][1] != None):
            scores[i][j][1] = res[0][1]

for i, row in enumerate(scores):  
    wins = 0
    loss = 0
    for value in row:
      if (value[0] > 0):
        wins += value[0]
      elif (value[0] < 0):
        loss += value[0]
    print(f'{agents[i]}|{wins}|{loss}')



rock_agent.py|102.0|-197.0
paper_agent.py|102.0|-196.0
scissors_agent.py|102.0|-198.0
random_agent.py|0|0
copy_opponent.py|0|0
reactionary.py|0|0
statistical.py|297.0|0


statistical.py показала лучший результат
